In [1]:
import os
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

from scipy.special import expit 

import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as pyo
import plotly.io as pio
from plotly.subplots import make_subplots

pyo.init_notebook_mode(connected = True)


from sklearn.metrics import roc_auc_score, f1_score
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, RobustScaler, GroupKFold
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import (
    GridSearchCV,
    train_test_split,
    cross_val_score,
    GroupShuffleSplit,
    permutation_test_score,
    StratifiedKFold,
    KFold,
    cross_validate
)

from tqdm import tqdm 

from sklearn.ensemble import ExtraTreesClassifier, RandomForestRegressor

import optuna
from merf import MERF
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold


import sys
sys.path.insert(0, '../')
from utils import multivariate_classifier, correct_name_markers, plot_univariate
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

ImportError: cannot import name 'GroupKFold' from 'sklearn.preprocessing' (/home/nicobruno/anaconda3/lib/python3.10/site-packages/sklearn/preprocessing/__init__.py)

In [4]:
# plotting parameters
grey = "#21201F"
green = "#9AC529"
lblue = "#42B9B2"
pink = "#DE237B"
orange = "#F38A31"

nt_colors = [green, lblue, pink, orange]

plt.style.use("ggplot")
fig_width = 12  # width in inches
fig_height = 9  # height in inches
fig_size = [fig_width, fig_height]
plt.rcParams["figure.figsize"] = fig_size
plt.rcParams["figure.autolayout"] = True

sns.set(
    style="white",
    context="notebook",
    font_scale=1,
    rc={
        "axes.labelcolor": grey,
        "text.color": grey,
        "axes.edgecolor": grey,
        "xtick.color": grey,
        "ytick.color": grey,
    },
)

sns.set_palette(sns.color_palette
(nt_colors))

# Load Data
Loads data from the computed markers. From `Data` directory

In [5]:
data_path = "../../Data/"
results_path = "../../Results/"
fig_path = "../../Results/Figs/"

df = pd.read_csv(os.path.join(data_path, 'all_markers.csv'), index_col = 0)

In [6]:
markers = ['wSMI_1', 'wSMI_2', 'wSMI_4', 'wSMI_8', 'p_e_1', 'p_e_2',
       'p_e_4', 'p_e_8', 'k', 'se','msf', 'sef90', 'sef95', 'b', 'b_n', 'g',
       'g_n', 't', 't_n', 'd', 'd_n', 'a_n', 'a', 'CNV', 'P1', 'P3a', 'P3b',]

erps =['CNV', 'P1', 'P3a', 'P3b']

df_markers = (df
              .query("stimuli == 'go'") # only go trials
              .query("correct == 'correct'") #only correct trials
              .query('prev_trial < 5') # only last 5 trials before each probe. 
              .drop(['stimuli', 'correct', 'prev_trial', 'label', 'events',  'epoch_type', 'preproc', 'ft', 'ft_n'], axis = 1) # drop unnecessary columns
              .query("mind in ['on-task','dMW', 'sMW']") # only mind wandering and on-task trials
              .groupby(['segment', 'participant']).filter(lambda x: len(x) > 1) # drop participants with less than 2 trials per segment
             )

# By segment Mulivariate analysis

## On-task Vs Mind- Wandering
This can only be performed for PC probes  as they are the only ones with On-task reports.


Prepares the dataframe for the analysis

In [34]:
variance = lambda x: np.std(x)/np.mean(x)

agg_dict = {k: ["mean", 'std'] for k in markers}
agg_dict.update({k: "first" for k in df_markers.drop(markers, axis=1).columns})

df_mind = (
    df_markers.query("probe == 'PC'")
    .groupby(["segment", "participant"], as_index=False)
    .agg(agg_dict)
    #     .query("mind != 'sMW'") #if you want to test against just one of the mw
    .assign(mind2=lambda df: np.where(df.mind == "on-task", "on-task", "mw"))
)

#### Use latex command for nmaes###
##it slow downs the computer, just for final figures.

df_mind = correct_name_markers(df_mind) #correct names for latex

df_mind.columns = df_mind.columns.map("$_{".join).map(lambda x: x + '}$').map(lambda x: x.replace('$$', ''))

df_mind  = (df_mind
            .rename(columns = {'participant$_{first}$':'participant', 'probe$_{first}$':'probe', 'mind$_{first}$':'mind', 'segment$_{first}$':'segment', 'mind2$_{}$':'mind2'})
#             .query("mind != 'dMW'") #if you want to test against just one of the mw            
            .drop(['probe', 'mind', 'segment'], axis = 1) 
            )

df_mind['mind2_numeric'] = (df_mind['mind2'] == 'mw').astype(int)

In [89]:
# Prepare data
X = df_mind.drop(['mind2', 'mind2_numeric', 'participant',], axis=1)
Z = np.ones((X.shape[0], 1))  # Random effects design matrix
clusters = df_mind['participant']
y = df_mind['mind2_numeric']

# # Add a constant term to the predictors
# X_const = add_constant(X)
# # Calculate VIF for each predictor variable
# vif_data = pd.DataFrame()
# vif_data["feature"] = X_const.columns
# vif_data["VIF"] = [variance_inflation_factor(X_const.values, i) for i in range(len(X_const.columns))]
# selected_features = vif_data[vif_data["VIF"] <= 10]["feature"].tolist()
# if 'const' in selected_features:
#     selected_features.remove('const')

# X = X[selected_features]

# Initialize lists to store AUC and permutation scores
actual_auc_scores = []
perm_auc_scores_all = []
optimal_cutoffs = []

# Perform Stratified KFold Cross-Validation
for train_index, test_index in tqdm(skf.split(X, y), desc="KFold Iteration"):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    clusters_train, clusters_test = clusters.iloc[train_index], clusters.iloc[test_index]
    
    # Initialize and train MERF
    merf = MERF(fixed_effects_model=RandomForestRegressor(
                n_estimators=100,  # Reduced number of trees
                max_depth=10,  # Added max depth
                min_samples_leaf=1,  # Minimum samples per leaf
                max_features=0.7,  # Feature sampling
                random_state=42,
                n_jobs=-1
    ),
    gll_early_stop_threshold=0.5,  # Early stopping
    max_iterations=50
)
    merf.fit(X_train, Z[train_index], clusters_train, y_train)
    
    
    y_pred = merf.predict(X_test, Z[test_index], clusters_test)

    # Convert continuous outputs to probabilities
    y_pred_proba = expit(y_pred)
    
    # Find optimal cutoff point based on F1 score
    thresholds = np.linspace(0, 1, 100)
    f1_scores = [f1_score(y_test, (y_pred_proba > t).astype(int)) for t in thresholds]
    optimal_threshold = thresholds[np.argmax(f1_scores)]
    optimal_cutoffs.append(optimal_threshold)
    
    # Thresholding to get class labels
    y_pred_class = (y_pred_proba > optimal_threshold).astype(int)
    
    # Compute AUC for probabilities
    auc = roc_auc_score(y_test, y_pred_proba)
    actual_auc_scores.append(auc)
    
    # Initialize list to store permutation AUC scores for this fold
    perm_auc_scores = []

    # Permutation test (500 permutations)
    # for i in tqdm(range(500), desc="Permutation Test"):
    #     y_perm = np.random.permutation(y_train)
        
    #     # Train MERF with permuted labels
    #     merf.fit(X_train, Z[train_index], clusters_train, y_perm)
        
    #     # Make predictions and convert to probabilities
    #     y_pred_perm = merf.predict(X_test, Z[test_index], clusters_test)
    #     y_pred_perm_proba = expit(y_pred_perm)
        
    #     # Compute AUC
    #     perm_auc = roc_auc_score(y_test, y_pred_perm_proba)
    #     perm_auc_scores.append(perm_auc)

# Create DataFrames
df_forest_mind = pd.DataFrame({'auc': actual_auc_scores, 'Model': 'Actual'})
# df_perm_mind = pd.DataFrame({'auc': perm_auc_scores_all, 'Model': 'Permutation'})

# Calculate p-value
# pvalue_mw = np.mean(df_perm_mind['auc'] >= np.mean(df_forest_mind['auc']))

# Save DataFrames to CSV
df_forest_mind.to_csv(os.path.join(results_path, 'merf_mvpa_mind.csv'), index=False)
# df_perm_mind.to_csv(os.path.join(results_path, 'merf_mvpa_mind_perm.csv'), index=False)


KFold Iteration: 0it [00:00, ?it/s]INFO     [merf.py:307] Training GLL is -269.7654701367638 at iteration 1.
INFO     [merf.py:307] Training GLL is -412.0884442980513 at iteration 2.
INFO     [merf.py:307] Training GLL is -430.3081418289836 at iteration 3.
INFO     [merf.py:321] Gll -430.3081418289836 less than threshold 0.04421307557402542, stopping early ...
KFold Iteration: 1it [00:03,  3.71s/it]INFO     [merf.py:307] Training GLL is -269.8988346431124 at iteration 1.
INFO     [merf.py:307] Training GLL is -408.1281977115593 at iteration 2.
INFO     [merf.py:307] Training GLL is -426.707241732485 at iteration 3.
INFO     [merf.py:321] Gll -426.707241732485 less than threshold 0.045522568950396865, stopping early ...
KFold Iteration: 2it [00:06,  3.32s/it]INFO     [merf.py:307] Training GLL is -270.84728836267016 at iteration 1.
INFO     [merf.py:307] Training GLL is -416.205994382401 at iteration 2.
INFO     [merf.py:307] Training GLL is -429.18424368747947 at iteration 3.
INFO     

In [9]:
df_forest_mind = pd.read_csv(os.path.join(results_path, 'merf_mvpa_mind.csv'))

print(df_forest_mind.auc.mean())

fig =px.violin(df_forest_mind, x = 'auc', box = True, points = 'all',template = "plotly_white", color_discrete_sequence = [pink], labels = {'auc': 'AUC'})
fig.add_vline(x=0.5, line_width=3, line_dash="dash", line_color="grey")

fig.update_layout(
#     title = f'Mean AUC = {df_forest_mind.auc.mean()}',
    autosize=False,
    width=800,
    height=800,
    yaxis = {'title': 'Whole model',
            'showticklabels': True,
            'tickmode': 'linear',},
    xaxis ={
             'range':[0.2, 1]
        }
)
fig.show()
fig.write_image(os.path.join(fig_path, 'multivariate_merf_cv_mind.png'))


0.6106639146886825


In [ ]:
df = pd.concat([df_perm_mind, df_forest_mind])
fig = px.histogram(df_perm_mind, x='auc', color="Model",nbins = 25,  color_discrete_sequence = [pink])

fig.add_vline(x=df_forest_mind.auc.mean(), line_width=3, line_dash="dash", line_color="grey")

fig.add_annotation(x=df_forest_mind.auc.mean() + 0.1, y=50,
            text=f"Score on original labels: <br>Mean AUC: {df_forest_mind.auc.mean():.3f}<br>p = {pvalue_mind:.3f}",
            showarrow=False,
            arrowhead=1,
            align = 'left',
            )


fig.update_layout(
#     title = f'Mean AUC = {df_forest_mind.auc.mean()}',
    autosize=False,
    template = "plotly_white",
    font=dict(
        family="Times new roman",
        size=20,
        color="black"
    ),
    width=800,
    height=800,
    bargap=0.05,
    yaxis = {'title': '',
             'tickfont': {"size": 20},
    },
    xaxis ={'title':'CV AUC',
             'range':[0.25, 0.75],
             'tickfont':{"size": 20}
        }
)
fig.show()

fig.write_image(os.path.join(fig_path, 'multivariate_perm_mind.png'))
fig.write_image(os.path.join(fig_path, 'multivariate_perm_mind.svg'))

In [ ]:
top_10 = feat_import_mind.sort_values(by = 'mean_importance').tail(10)
fig = px.scatter(top_10,x = 'mean_importance', y = 'features', template = "plotly_white",
                  color_discrete_sequence = [pink],

                 category_orders = {'significant': ['p > 0.05','p < 0.05 uncorrected', 'p < 0.05 FDR corrected']}, 
                 labels = {'value':'Feature importance', 'features': 'Markers'}

                )

fig.update_traces(marker=dict(size = 8))

fig.update_layout(
    width=650,
    height=900,
#     autosize = True, 
    template = 'plotly_white',
        font=dict(
        family="Times new roman",
        size=20,
        color="black"
    ),
    xaxis = dict(
            visible=True,
            # range = [0.37,0.63], 
            tickfont = {"size": 20},
            title = 'Feature Importance'
        ),
    yaxis = dict(
        tickfont = {"size": 20},
        # autorange = False,    
        automargin = True,
        # range = [-1,len(feat_import_mw)],
        dtick = 1
        ),
    showlegend=True, 

)

fig.show()
# pio.write_json(fig, 'Figs/univariate_roc_mw_segment.plotly')
fig.write_image(os.path.join(fig_path, 'feat_importance_mind.png'))
fig.write_image(os.path.join(fig_path, 'feat_importance_mind.svg'))

## dMW Vs sMW
This will be only performed in SC as they have more trials

In [ ]:
agg_dict = {k:['mean','std'] for k in markers }
agg_dict.update({k:'first' for k in df_markers.drop(markers, axis=1).columns})

df_mw = (
    df_markers
    .query("probe == 'SC'")
    .query("mind != 'on-task'")
    .groupby(['segment', 'participant'], as_index = False).agg(agg_dict)
)

# df_mw.columns = df_mw.columns.map("_".join)

# df_mw  = (df_mw
#             .rename(columns = {'participant_first':'participant', 'probe_first':'probe', 'mind_first':'mind', 'segment_first':'segment'})
#             .drop(['participant', 'probe', 'segment'], axis = 1) 
#            )



#### Use latex command for nmaes###
####it slow downs the computer, just for final figures.

df_mw = correct_name_markers(df_mw)

df_mw.columns = df_mw.columns.map("$_{".join).map(lambda x: x + '}$').map(lambda x: x.replace('$$', ''))

df_mw  = (df_mw
            .rename(columns = {'participant$_{first}$':'participant', 'probe$_{first}$':'probe', 'mind$_{first}$':'mind', 'segment$_{first}$':'segment', 'mind$_{}$':'mind'})
#             .query("mind != 'dMW'") #if you want to test against just one of the mw   
            .drop([ 'probe',  'segment'], axis = 1)

           )
df_mw['mind_numeric'] = (df_mw['mind'] == 'sMW').astype(int)


In [91]:
# Prepare data
X = df_mw.drop(['mind', 'mind_numeric', 'participant',], axis=1)
Z = np.ones((X.shape[0], 1))  # Random effects design matrix
clusters = df_mw['participant']
y = df_mw['mind_numeric']

# # Add a constant term to the predictors
# X_const = add_constant(X)
# # Calculate VIF for each predictor variable
# vif_data = pd.DataFrame()
# vif_data["feature"] = X_const.columns
# vif_data["VIF"] = [variance_inflation_factor(X_const.values, i) for i in range(len(X_const.columns))]
# selected_features = vif_data[vif_data["VIF"] <= 50]["feature"].tolist()
# if 'const' in selected_features:
#     selected_features.remove('const')

# X = X[selected_features]

# Initialize lists to store AUC and permutation scores
actual_auc_scores = []
perm_auc_scores_all = []
optimal_cutoffs = []

# Perform Stratified KFold Cross-Validation
for train_index, test_index in tqdm(skf.split(X, y), desc="KFold Iteration"):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    clusters_train, clusters_test = clusters.iloc[train_index], clusters.iloc[test_index]
    
    # Initialize and train MERF
    merf = MERF(fixed_effects_model=RandomForestRegressor(
                n_estimators=100,  # Reduced number of trees
                max_depth=10,  # Added max depth
                min_samples_leaf=1,  # Minimum samples per leaf
                max_features=0.7,  # Feature sampling
                random_state=42,
                n_jobs=-1
    ),
    gll_early_stop_threshold=0.5,  # Early stopping
    max_iterations=50
)
    merf.fit(X_train, Z[train_index], clusters_train, y_train)
    
    
    y_pred = merf.predict(X_test, Z[test_index], clusters_test)

    # Convert continuous outputs to probabilities
    y_pred_proba = expit(y_pred)
    
    # Find optimal cutoff point based on F1 score
    thresholds = np.linspace(0, 1, 100)
    f1_scores = [f1_score(y_test, (y_pred_proba > t).astype(int)) for t in thresholds]
    optimal_threshold = thresholds[np.argmax(f1_scores)]
    optimal_cutoffs.append(optimal_threshold)
    
    # Thresholding to get class labels
    y_pred_class = (y_pred_proba > optimal_threshold).astype(int)
    
    # Compute AUC for probabilities
    auc = roc_auc_score(y_test, y_pred_proba)
    actual_auc_scores.append(auc)
    
    # Initialize list to store permutation AUC scores for this fold
    perm_auc_scores = []

    # Permutation test (500 permutations)
    # for i in tqdm(range(500), desc="Permutation Test"):
    #     y_perm = np.random.permutation(y_train)
        
    #     # Train MERF with permuted labels
    #     merf.fit(X_train, Z[train_index], clusters_train, y_perm)
        
    #     # Make predictions and convert to probabilities
    #     y_pred_perm = merf.predict(X_test, Z[test_index], clusters_test)
    #     y_pred_perm_proba = expit(y_pred_perm)
        
    #     # Compute AUC
    #     perm_auc = roc_auc_score(y_test, y_pred_perm_proba)
    #     perm_auc_scores.append(perm_auc)

# Create DataFrames
df_forest_mw = pd.DataFrame({'auc': actual_auc_scores, 'Model': 'Actual'})
# df_perm_mw = pd.DataFrame({'auc': perm_auc_scores_all, 'Model': 'Permutation'})
print(df_forest_mw.auc.mean())

# Calculate p-value
# pvalue_mw = np.mean(df_perm_mind['auc'] >= np.mean(df_forest_mind['auc']))

# Save DataFrames to CSV
df_forest_mw.to_csv(os.path.join(results_path, 'merf_mvpa_mw.csv'), index=False)
# df_perm_mw.to_csv(os.path.join(results_path, 'merf_mvpa_mw_perm.csv'), index=False)


KFold Iteration: 0it [00:00, ?it/s]INFO     [merf.py:307] Training GLL is -969.4131274490296 at iteration 1.
INFO     [merf.py:307] Training GLL is -1077.5451276039635 at iteration 2.
INFO     [merf.py:321] Gll -1077.5451276039635 less than threshold 0.11154377539685141, stopping early ...
KFold Iteration: 1it [00:07,  7.28s/it]INFO     [merf.py:307] Training GLL is -943.0821291209862 at iteration 1.
INFO     [merf.py:307] Training GLL is -1068.771242785236 at iteration 2.
INFO     [merf.py:321] Gll -1068.771242785236 less than threshold 0.13327483342452912, stopping early ...
KFold Iteration: 2it [00:12,  5.80s/it]INFO     [merf.py:307] Training GLL is -942.9294336884783 at iteration 1.
INFO     [merf.py:307] Training GLL is -1071.0456007313978 at iteration 2.
INFO     [merf.py:321] Gll -1071.0456007313978 less than threshold 0.13587036576190503, stopping early ...
KFold Iteration: 3it [00:16,  5.01s/it]INFO     [merf.py:307] Training GLL is -951.566555639009 at iteration 1.
INFO     

0.671535390199637


In [11]:
df_forest_mw = pd.read_csv(os.path.join(results_path, 'merf_mvpa_mw.csv'))

print(df_forest_mw.auc.mean())

fig =px.violin(df_forest_mw, x = 'auc', box = True, points = 'all',template = "plotly_white", color_discrete_sequence = [lblue], labels = {'auc': 'AUC'})
fig.add_vline(x=0.5, line_width=3, line_dash="dash", line_color="grey")

fig.update_layout(
#     title = f'Mean AUC = {df_forest_mind.auc.mean()}',
    autosize=False,
    width=800,
    height=800,
    yaxis = {'title': 'Whole model',
            'showticklabels': True,
            'tickmode': 'linear',},
    xaxis ={
             'range':[0.2, 1]
        }
)
fig.show()

fig.write_image(os.path.join(fig_path, 'multivariate_merf_cv_mw.png'))


0.671535390199637
